# 1D Neural Network Position Model

The goal of this notebook is to replicate the results of the 1D neural network

### Hyperparameters

In [1]:
HIDDEN_SIZES = [5, 5, 5]
EPOCHS = 25000

### Load Data

In [ ]:
%store -r data_1d

df = data_1d
sideways = df.T

X, Y = np.array(sideways[:10]), np.array(sideways.ix['label']).astype(int)

assert str(X.dtype) == 'float64'

### Train

In [ ]:
from nn.deep.model import NeuralNetwork

nn = NeuralNetwork(X, Y, Hs=HIDDEN_SIZES, C=2, learning_rate=.1, regularizer=.001, gradient_checking=False, inspect=True)

def states(iters):
    for _ in range(iters):
        nn.learn()
        yield nn.info
        
states = list(states(EPOCHS))

### Visualize

Notice the plots are linear because of the following recurrence relation:

$$v_t = x_{t+1} - x_t$$

This formula follows by definition of $x_t$:

$$x_t = x_{t-1} + v_{t-1}$$

Intuitively, as the difference between $x$ points gets smaller, the difference between $y$ points get smaller **by the same amount**

In [ ]:
from ipywidgets import interact, fixed

@interact(X=fixed(X), Y=fixed(Y), view_index=(0, len(states)-1), states=fixed(states))
def plot(X, Y, view_index, states):
    """View decision boundary over time
    
    We have the state of the model (i.e. its parameters) after every time step.
    We want to see how the model learns over time. Hence create a one-off instance
    of a softmax with its old parameters and ask it to make a bunch of predictions!
    """
    s = states[view_index]
    nn = NeuralNetwork(X, Y, Hs=HIDDEN_SIZES, C=2, params=s.params)
    
    # Plotting
    colormap = np.array(['r', 'b'])

    # Plot the gold standard in a plot right next to it
    axes = df.plot(kind='scatter', x='pos_avg', y='shot_number', c=colormap[df['label']], title='Ground Truth', s=0)
    axes.quiver(df['pos_avg'], df['shot_number'], df['vel_avg'], np.zeros_like(df['vel_avg']), color=colormap[df['label']], scale_units='xy', angles='xy', scale=2.5, width=.0022)

    # Predictions
    df['predictions'] = nn.predict(X)
    axes = df.plot(kind='scatter', x='pos_avg', y='shot_number', c=colormap[df['predictions']], title='Predictions', s=0)
    axes.quiver(df['pos_avg'], df['shot_number'], df['vel_avg'], np.zeros_like(df['vel_avg']), color=colormap[df['predictions']], scale_units='xy', angles='xy', scale=2.5, width=.0022)

    # Misclassified examples
    df['correct'] = df['label'] == df['predictions']
    axes = df.plot(kind='scatter', x='pos_avg', y='shot_number', c=colormap[df['correct']], title='Missclassified', s=0)
    axes.quiver(df['pos_avg'], df['shot_number'], df['vel_avg'], np.zeros_like(df['vel_avg']), color=colormap[df['correct']], scale_units='xy', angles='xy', scale=2.5, width=.0022)

### Plot Losses

In [ ]:
def loss_averages():
    for state in states:
        yield state.gradients['loss'].mean()
        
loss_avgs = list(loss_averages())

pd.DataFrame(loss_avgs, columns=['loss']).plot()

In [ ]:
df['correct'].sum() / len(df)